# Parameters

<img src="./images/parameter.jpg" width="30%">

Proste lokalne api do trzymania *parametrów* -- głównie ustawień programów w trakcie uruchamiania

In [1]:



import rclpy
from rclpy.node import Node

from std_msgs.msg import String
from rclpy.parameter import Parameter
from rcl_interfaces.msg import SetParametersResult



class MinimalNode(Node):

    def __init__(self):
        super().__init__('test_params_rclpy')
        self.declare_parameter('my_str', rclpy.Parameter.Type.STRING) 
        self.declare_parameter('my_int', rclpy.Parameter.Type.INTEGER) 
        self.declare_parameter('my_double_array', rclpy.Parameter.Type.DOUBLE_ARRAY)
        param_str = Parameter('my_str', Parameter.Type.STRING, 'Set from code')
        param_int = Parameter('my_int', Parameter.Type.INTEGER, 12)
        param_double_array = Parameter('my_double_array', Parameter.Type.DOUBLE_ARRAY, [1.1, 2.2])
        self.add_on_set_parameters_callback(self.parameter_callback)
        self.set_parameters([param_str, param_int, param_double_array])
    def parameter_callback(self,params):
        print(params)
        for param in params:
            print(param.name, param.value)
            if param.name == "my_str":
                print("wow, you changed my_str")
        return SetParametersResult(successful=True)
try:
    rclpy.init()

    minimal_node = MinimalNode()
except RuntimeError:
    pass

my_str Set from code
wow, you changed my_str
my_int 12
my_double_array [1.1, 2.2]


In [6]:
!ros2 param list

/amcl:
  /bond_disable_heartbeat_timeout
  alpha1
  alpha2
  alpha3
  alpha4
  alpha5
  always_reset_initial_pose
  base_frame_id
  beam_skip_distance
  beam_skip_error_threshold
  beam_skip_threshold
  do_beamskip
  first_map_only
  global_frame_id
  initial_pose.x
  initial_pose.y
  initial_pose.yaw
  initial_pose.z
  lambda_short
  laser_likelihood_max_dist
  laser_max_range
  laser_min_range
  laser_model_type
  map_topic
  max_beams
  max_particles
  min_particles
  odom_frame_id
  pf_err
  pf_z
  qos_overrides./clock.subscription.depth
  qos_overrides./clock.subscription.durability
  qos_overrides./clock.subscription.history
  qos_overrides./clock.subscription.reliability
  qos_overrides./tf.publisher.depth
  qos_overrides./tf.publisher.durability
  qos_overrides./tf.publisher.history
  qos_overrides./tf.publisher.reliability
  recovery_alpha_fast
  recovery_alpha_slow
  resample_interval
  robot_model_type
  save_pose_rate
  scan_topic
  set_initial_pose
  sigma_hit
  tf_broadca

# W ROS2 każdy node ma (łatwy) dostęp do swoich parametrów. 
Czyjeś parametry dostępne są przez service

In [2]:
minimal_node.get_parameter("my_str").value

'Set from code'

In [4]:
minimal_node.set_parameters([Parameter('my_int', Parameter.Type.INTEGER, 55)])

my_int 55


[rcl_interfaces.msg.SetParametersResult(successful=True, reason='')]

In [8]:
import subscription_threaded

threader = subscription_threaded.ThreadedSpinner(minimal_node)

In [9]:
threader.spin_in_thread()

# Możemy zmieniać parametery z poziomu lini komend

In [11]:
!ros2 param set /test_params_rclpy my_str "zenon"

my_str zenon
wow, you changed my_str
Set parameter successful


In [13]:
minimal_node.get_parameter("my_str").value

'zenon'